In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from keras.preprocessing import image

In [ ]:
train_datagen=image.ImageDataGenerator(
    rescale=1./255,horizontal_flip=True,zoom_range=0.2,shear_range=0.2
)
train_data=train_datagen.flow_from_directory('/content/drive/MyDrive/ColabNotebooks/Covid19-dataset/train',target_size=(256,256),batch_size=10,
                                             class_mode='binary')

Found 181 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
test_datagen=image.ImageDataGenerator(
    rescale=1./255
)
test_data=test_datagen.flow_from_directory('/content/drive/MyDrive/ColabNotebooks/Covid19-dataset/test',target_size=(256,256),batch_size=10,
                                             class_mode='binary')

Found 46 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(256,256,3)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))



model.add(Flatten())

model.add(Dense(units=50,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                    

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [ ]:
model.fit_generator(train_data,epochs=16,validation_steps=2,validation_data=test_data,callbacks=[earlystop])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/16
19/19 [==============================] - 118s 6s/step - loss: 1.0683 - accuracy: 0.5801 - val_loss: 0.6068 - val_accuracy: 0.6000
Epoch 2/16
19/19 [==============================] - 74s 4s/step - loss: 0.5309 - accuracy: 0.7293 - val_loss: 0.4278 - val_accuracy: 0.9500
Epoch 3/16
19/19 [==============================] - 72s 4s/step - loss: 0.3333 - accuracy: 0.8343 - val_loss: 0.1612 - val_accuracy: 1.0000
Epoch 4/16
19/19 [==============================] - 75s 4s/step - loss: 0.2950 - accuracy: 0.8674 - val_loss: 0.2907 - val_accuracy: 0.9500
Epoch 5/16
19/19 [==============================] - 72s 4s/step - loss: 0.3338 - accuracy: 0.8619 - val_loss: 0.1911 - val_accuracy: 0.9500
Epoch 6/16
19/19 [==============================] - 71s 4s/step - loss: 0.3428 - accuracy: 0.8840 - val_loss: 0.2871 - val_accuracy: 0.9000
Epoch 7/16
19/19 [==============================] - 72s 4s/step - loss: 0.2893 - accuracy: 0.8453 - val_loss: 0.1255 - val_accuracy: 1.0000
Epoch 8/16
19/19 [=

In [ ]:
acc=model.evaluate(test_data,steps=len(test_data),verbose=2)
print('%.2f'%(acc[1]*100))

5/5 - 5s - loss: 0.1704 - accuracy: 0.9565 - 5s/epoch - 938ms/step
95.65


In [ ]:
from tensorflow.keras.models import load_model
model.save('model1.h5')

In [ ]:
model12=load_model('model1.h5')

In [ ]:
path='/content/drive/MyDrive/ColabNotebooks/Covid19-dataset/test/Covid/COVID-00003b.jpg'
img=image.load_img(path,target_size=(256,256))
img=image.img_to_array(img)/256
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
model12.predict(img)

array([[0.05850682]], dtype=float32)

In [ ]:
path='/content/drive/MyDrive/ColabNotebooks/Covid19-dataset/test/Normal/0107.jpeg'
img=image.load_img(path,target_size=(256,256))
img=image.img_to_array(img)/256
img=np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
model12.predict(img)

array([[0.9722514]], dtype=float32)